In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


dataset=pd.read_csv("prep.csv")
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [41]:
df = pd.get_dummies(dataset,dtype=int,drop_first=True)

In [69]:
x =df.iloc[:,0:24].values
featurenames = df.iloc[:,0:24].columns

In [43]:
y=df.iloc[:,24].values

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [45]:
def standardscaler(x_train,x_test):
    sc=StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train,x_test

In [87]:
def pca(x_train,x_test,n):
    pca = PCA(n_components = n)
    x_train = pca.fit_transform(x_train)
    x_test = pca.transform(x_test)
    print(pca.components_)
    most_influential_features = np.argmax(np.abs(pca.components_), axis=1) # This will give the maximum PCA component index [10,6] something for 2 PCA component
    print(most_influential_features)
    selected_features = featurenames[most_influential_features]
    explained_variance = pca.explained_variance_ratio_
    return x_train,x_test,explained_variance,selected_features

In [47]:
def cm_prediction(classifier,x_test):
    y_pred = classifier.predict(x_test)
    
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)
    
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    Accuracy = accuracy_score(y_test,y_pred)
    report=classification_report(y_test,y_pred)
    return classifier,Accuracy,report,x_test,y_test,cm

In [48]:
def accuracy_table(acclog,accsvml,accknn,accnav,accdes,accrf):
    dataframe  = pd.DataFrame(index=[f"Dimensions: {len(explained_variance)}"],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [49]:
def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(x_train,y_train)
    classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,Accuracy,report,x_test,y_test,cm

In [50]:
def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return  classifier,Accuracy,report,x_test,y_test,cm

In [51]:
def svm_NL(x_train,y_train,x_test):        
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(x_train, y_train)
        classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
        return  classifier,Accuracy,report,x_test,y_test,cm

In [52]:
def Navie(x_train,y_train,x_test):       
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(x_train, y_train)
        classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
        return  classifier,Accuracy,report,x_test,y_test,cm   

In [53]:
def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5,metric = "minkowski",p=2)
    classifier.fit(x_train,y_train)
    classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,Accuracy,report,x_test,y_test,cm

In [54]:
def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion="entropy",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,Accuracy,report,x_test,y_test,cm

In [55]:
def random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators=10,criterion="entropy",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,Accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,Accuracy,report,x_test,y_test,cm

In [91]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)
x_train,x_test = standardscaler(x_train,x_test)
x_train,x_test,explained_variance,selected_features = pca(x_train,x_test,2);
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
print(featurenames)
print(selected_features)

[[ 0.13618365  0.14813497  0.26850783  0.17014439  0.19508084  0.25872023
   0.20329875 -0.18104895  0.06447555 -0.32121897 -0.32340318  0.08278028
  -0.28547554 -0.17557578  0.15796509  0.11414305  0.06110781 -0.15557921
  -0.2392516   0.17462042  0.15482696  0.27335967  0.25743607  0.17214565]
 [-0.02984445  0.03687412 -0.25870828 -0.26088397 -0.25231969  0.28852262
   0.47005872 -0.36523946  0.15595631 -0.1120537  -0.10754389 -0.20571942
  -0.13593846  0.02736469 -0.11654992 -0.08000814 -0.10101889  0.13414689
   0.25592213 -0.28556339 -0.21548171  0.07477026 -0.01802303  0.06265111]]
[10  6]
Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes'],
      dtype='object')
Index(['pcv', 'sc'], dtype='object')


In [92]:
classifier,Accuracy,report,x_test,y_test,cm=logistic(x_train,y_train,x_test)
acclog.append(Accuracy)

classifier,Accuracy,report,x_test,y_test,cm=svm_linear(x_train,y_train,x_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,x_test,y_test,cm=svm_NL(x_train,y_train,x_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,x_test,y_test,cm=knn(x_train,y_train,x_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,x_test,y_test,cm=Navie(x_train,y_train,x_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,x_test,y_test,cm=Decision(x_train,y_train,x_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,x_test,y_test,cm=random(x_train,y_train,x_test)  
accrf.append(Accuracy)

C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
accuracy_table(acclog,accsvml,accknn,accnav,accdes,accrf)

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Dimensions: 2,0.8,0.775,0.8,0.7625,0.8,0.6125,0.7125


In [21]:
accuracy_table(acclog,accsvml,accknn,accnav,accdes,accrf)

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Dimensions: 15,0.825,0.775,0.8125,0.725,0.7625,0.6625,0.7625


In [22]:
accuracy_table(acclog,accsvml,accknn,accnav,accdes,accrf)

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Dimensions: 15,0.825,0.775,0.8125,0.725,0.7625,0.6625,0.7625


In [23]:
accuracy_table(acclog,accsvml,accknn,accnav,accdes,accrf)

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Dimensions: 15,0.825,0.775,0.8125,0.725,0.7625,0.6625,0.7625


In [89]:
print(selected_features) #for 2 dimensions

Index(['pcv', 'sc'], dtype='object')
